In [1]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, AutoRegressiveBaseModel,AutoRegressiveBaseModelWithCovariates
from pytorch_forecasting.metrics import MAE,CrossEntropy
import os
import pandas as pd # type: ignore
import matplotlib.pyplot as plt # type: ignore
import numpy as np # type: ignore
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from pytorch_forecasting.models.nn import LSTM
from typing import Dict
from sklearn.model_selection import train_test_split

/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/pytorch_forecasting/models/base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
stream = pd.read_pickle(os.path.join('..','data','processed','stream_processed.pkl'),compression= 'zip')
meteo = pd.read_pickle(os.path.join('..','data','processed','meteo_processed.pkl'),compression = 'zip')
stream

measurement_location,Datum,15202300,15205501,15207507,15210206,15212008,15212700,15213500,15214003,15214604,15216009,15217908,15221009,15228008,15241006,15242304,15243001,15246000,15247002,15247501
0,2014-01-01,14.4,17.2,19.8,3.87,8.66,9.66,0.433,2.35,0.207,1.66,0.921,0.671,2.59,0.253,0.870,1.26,2.35,0.355,0.120
1,2014-01-02,14.4,17.0,19.3,3.78,8.52,9.46,0.435,2.51,0.204,1.62,0.909,0.664,2.57,0.253,0.870,1.23,2.32,0.361,0.131
2,2014-01-03,14.1,16.6,19.3,3.78,8.40,9.42,0.429,2.61,0.201,1.60,0.907,0.641,2.58,0.253,0.871,1.23,2.30,0.362,0.130
3,2014-01-04,14.2,16.6,19.3,3.84,8.57,9.50,0.445,2.49,0.214,1.64,0.935,0.642,2.60,0.256,0.929,1.26,2.44,0.382,0.127
4,2014-01-05,16.4,18.4,20.6,4.36,9.94,10.80,0.545,2.86,0.267,1.80,1.130,0.690,3.10,0.300,1.150,1.61,3.14,0.561,0.175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,2023-12-28,79.7,102.0,120.0,28.50,46.20,57.10,1.900,17.30,0.721,10.30,6.070,3.920,12.20,0.786,3.270,9.05,12.50,2.020,0.741
3649,2023-12-29,71.4,82.5,97.7,26.00,41.90,51.30,1.710,15.60,0.801,9.85,5.480,3.470,10.90,0.653,2.890,7.23,9.97,1.750,0.619
3650,2023-12-30,64.0,71.4,86.3,24.40,38.60,46.90,1.530,14.70,0.835,9.40,4.900,3.100,9.77,0.557,2.550,5.92,8.54,1.470,0.514
3651,2023-12-31,59.1,63.0,77.3,22.20,35.30,42.50,1.410,13.10,0.669,8.83,4.480,2.670,8.92,0.492,2.330,5.40,7.66,1.290,0.458


In [3]:
stream = stream.iloc[:3652]
data = stream.join(meteo).drop(columns = 'Datum')
data['groups'] = 0
data['date'] = data.index
data['date']

0          0
1          1
2          2
3          3
4          4
        ... 
3647    3647
3648    3648
3649    3649
3650    3650
3651    3651
Name: date, Length: 3652, dtype: int64

In [4]:
data['15207507']

0        19.8
1        19.3
2        19.3
3        19.3
4        20.6
        ...  
3647    161.0
3648    120.0
3649     97.7
3650     86.3
3651     77.3
Name: 15207507, Length: 3652, dtype: float64

In [50]:
X_train = data[:int(len(data)*0.85)].reset_index(drop = True)
X_train['date'] = X_train.index
X_test = data[int(len(data)*0.85)+1:].reset_index(drop = True)
X_test['date'] = X_test.index

In [6]:
X_train['date']

0          0
1          1
2          2
3          3
4          4
        ... 
2551    2551
2552    2552
2553    2553
2554    2554
2555    2555
Name: date, Length: 2556, dtype: int64

In [7]:
test_data_with_covariates = pd.DataFrame(
    dict(
        # as before
        value=np.random.rand(30),
        group=np.repeat(np.arange(3), 10),
        time_idx=np.tile(np.arange(10), 3),
        # now adding covariates
        categorical_covariate=np.random.choice(["a", "b"], size=30),
        real_covariate=np.random.rand(30),
    )
).astype(
    dict(group=str)
)  # categorical covariates have to be of string type
test_data_with_covariates

,value,group,time_idx,categorical_covariate,real_covariate
0,0.988272,0,0,b,0.741365
1,0.287448,0,1,a,0.946431
2,0.459120,0,2,a,0.642342
3,0.846460,0,3,b,0.725537
4,0.761446,0,4,a,0.873613
5,0.796390,0,5,a,0.896259
6,0.908661,0,6,b,0.342410
7,0.499741,0,7,b,0.526982
8,0.988352,0,8,a,0.961440
9,0.933370,0,9,b,0.591897


In [65]:
max_pred_len = 1
training_cutoff = int(data["date"].max()*0.7) - max_pred_len
val_cutoff = int(data["date"].max()*0.85) - max_pred_len

training = TimeSeriesDataSet(
    data= data.iloc[: training_cutoff],
    target='15207507',#'15207507'
    group_ids=['groups'],#groups
    time_idx='date',#date
    max_encoder_length=364,
    min_prediction_length=1,
    max_prediction_length=max_pred_len,
    time_varying_unknown_reals=['15207507'],
    #lags = {'15207507' : [1]},
    time_varying_known_reals=data.columns[20:-1].values.tolist()#data.columns[20:-1].values.tolist()
)
# validation = training = TimeSeriesDataSet(
#     data= X_val.iloc[:training_cutoff],
#     target='15207507',#'15207507'
#     group_ids=['groups'],#groups
#     time_idx='date',#date
#     max_encoder_length=364,
#     min_prediction_length=1,
#     max_prediction_length=max_pred_len,
#     time_varying_unknown_reals=['15207507'],
#     #lags = {'real_covariate' : [1,2]},
#     time_varying_known_reals=data.columns[20:-1].values.tolist()#data.columns[20:-1].values.tolist()
# )
validation = TimeSeriesDataSet.from_dataset(training,data.iloc[: val_cutoff],min_prediction_idx=training_cutoff,stop_randomization=True)

train_dataloader = training.to_dataloader(train=True, batch_size=32, num_workers=2)
val_dataloader = validation.to_dataloader(train=False, batch_size=32, num_workers=2)


In [24]:
x,y = next(iter(training))

tensor([ 13.8000,  12.9000,  12.2000,  12.5000,  11.9000,  15.2000,  16.3000,
         14.4000,  13.4000,  11.5000,  12.2000,  13.5000,  14.7000,  13.2000,
         12.4000,  12.1000,  11.2000,  11.6000,  11.3000,  11.1000,  10.8000,
         11.4000,  13.8000,  12.4000,  12.5000,  13.8000,  13.1000,  11.6000,
         12.1000,  10.8000,  10.9000,  10.5000,  10.1000,   9.8800,   9.6100,
          9.2700,   9.4100,   9.4800,  10.4000,   9.7400,   9.2400,   8.9400,
         11.1000,  14.0000,  15.0000,  16.3000,  14.0000,  12.4000,  11.5000,
         11.3000,  11.2000,  10.2000,  14.1000,  17.8000,  11.5000,  10.5000,
         10.1000,   9.6600,   9.1600,   8.7800,   9.8100,  10.4000,  10.4000,
         11.4000,   9.9900,   9.9900,  10.6000,  12.7000,  10.9000,  10.7000,
          9.7800,   9.6200,   9.2900,   9.0500,   9.7900,   9.1900,   9.1400,
          8.8900,   8.4200,   8.9000,   8.8400,   9.4100,  11.2000,  10.3000,
         10.6000,   9.2800,   8.7900,   8.6800,   8.8300,   7.92

In [26]:
class LSTMModel(AutoRegressiveBaseModel):
    def __init__(
        self,
        input_size:int,
        target: str,
        target_lags: Dict[str, Dict[str, int]],
        n_layers: int,
        hidden_size: int,
        dropout: float = 0.1,
        **kwargs,
    ):
        # arguments target and target_lags are required for autoregressive models
        # even though target_lags cannot be used without covariates
        # saves arguments in signature to `.hparams` attribute, mandatory call - do not skip this
        self.save_hyperparameters()
        # pass additional arguments to BaseModel.__init__, mandatory call - do not skip this
        super().__init__(**kwargs)

        # use version of LSTM that can handle zero-length sequences
        self.lstm = LSTM(
            hidden_size=self.hparams.hidden_size,
            input_size=self.hparams.input_size,
            num_layers=self.hparams.n_layers,
            dropout=self.hparams.dropout,
            batch_first=True,
        )
        self.output_layer = nn.Linear(self.hparams.hidden_size, 1)

    def encode(self, x: Dict[str, torch.Tensor]):
        # we need at least one encoding step as because the target needs to be lagged by one time step
        # because we use the custom LSTM, we do not have to require encoder lengths of > 1
        # but can handle lengths of >= 1
        assert x["encoder_lengths"].min() >= 1
        input_vector = x["encoder_cont"].clone()
        # lag target by one
        input_vector[..., self.target_positions] = torch.roll(
            input_vector[..., self.target_positions], shifts=1, dims=1
        )
        input_vector = input_vector[:, 1:]  # first time step cannot be used because of lagging

        # determine effective encoder_length length
        effective_encoder_lengths = x["encoder_lengths"] - 1
        # run through LSTM network
        _, hidden_state = self.lstm(
            input_vector, lengths=effective_encoder_lengths, enforce_sorted=False  # passing the lengths directly
        )  # second ouput is not needed (hidden state)
        return hidden_state

    def decode(self, x: Dict[str, torch.Tensor], hidden_state):
        # again lag target by one
        input_vector = x["decoder_cont"].clone()
        input_vector[..., self.target_positions] = torch.roll(
            input_vector[..., self.target_positions], shifts=1, dims=1
        )
        # but this time fill in missing target from encoder_cont at the first time step instead of throwing it away
        last_encoder_target = x["encoder_cont"][
            torch.arange(x["encoder_cont"].size(0), device=x["encoder_cont"].device),
            x["encoder_lengths"] - 1,
            self.target_positions.unsqueeze(-1),
        ].T
        input_vector[:, 0, self.target_positions] = last_encoder_target

        if self.training:  # training mode
            lstm_output, _ = self.lstm(input_vector, hidden_state, lengths=x["decoder_lengths"], enforce_sorted=False)

            # transform into right shape
            prediction = self.output_layer(lstm_output)
            prediction = self.transform_output(prediction, target_scale=x["target_scale"])

            # predictions are not yet rescaled
            return prediction

        else:  # prediction mode
            target_pos = self.target_positions

            def decode_one(idx, lagged_targets, hidden_state):
                x = input_vector[:, [idx]]
                # overwrite at target positions
                x[:, 0, target_pos] = lagged_targets[-1]  # take most recent target (i.e. lag=1)
                lstm_output, hidden_state = self.lstm(x, hidden_state)
                # transform into right shape
                prediction = self.output_layer(lstm_output)[:, 0]  # take first timestep
                return prediction, hidden_state

            # make predictions which are fed into next step
            output = self.decode_autoregressive(
                decode_one,
                first_target=input_vector[:, 0, target_pos],
                first_hidden_state=hidden_state,
                target_scale=x["target_scale"],
                n_decoder_steps=input_vector.size(1),
            )

            # predictions are already rescaled
            return output

    def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        hidden_state = self.encode(x)  # encode to hidden state
        output = self.decode(x, hidden_state)  # decode leveraging hidden state

        return self.to_network_output(prediction=output)



model = LSTMModel.from_dataset(training,input_size = 96,learning_rate = 0.001, n_layers=10, hidden_size=64,loss = MAE(),dropout = 0.2, optimizer = 'adamw')

/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [27]:
trainer = pl.Trainer(
    accelerator="auto",
    max_epochs=10,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [28]:
# fit network
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)


  | Name            | Type       | Params
-----------------------------------------------
0 | loss            | MAE        | 0     
1 | logging_metrics | ModuleList | 0     
2 | lstm            | LSTM       | 340 K 
3 | output_layer    | Linear     | 65    
-----------------------------------------------
341 K     Trainable params
0         Non-trainable params
341 K     Total params
1.364     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 9: 100%|██████████| 56/56 [07:41<00:00,  0.12it/s, v_num=4, train_loss_step=3.360, val_loss=2.490, train_loss_epoch=3.050]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 56/56 [07:43<00:00,  0.12it/s, v_num=4, train_loss_step=3.360, val_loss=2.490, train_loss_epoch=3.050]


In [29]:
best_model_path = trainer.checkpoint_callback.best_model_path
best_lstm = LSTMModel.load_from_checkpoint(best_model_path)

/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.


In [68]:
test = TimeSeriesDataSet.from_dataset(training,data,min_prediction_idx = val_cutoff,stop_randomization=True)
test_dataloader = test.to_dataloader(train=False, batch_size=55, num_workers=2)

In [70]:
predictions = best_lstm.predict(test_dataloader,return_y=True,trainer_kwargs=dict(accelerator="cpu"))


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.


In [93]:
predictions.output.shape

torch.Size([550, 1])

In [111]:
i = 20
print(predictions.y[0].view(-1, 1)[i],predictions.output[i])

tensor([26.]) tensor([13.1448])


In [114]:
def kge(inputs,targets):
    r = torch.corrcoef(torch.stack((inputs.squeeze(),targets)))[0][1]
    alpha = inputs.std()/targets.std()
    beta = inputs.mean()/targets.mean()
    kge = torch.sqrt(torch.square(r-1) + torch.square(alpha-1) + torch.square(beta-1))
    return kge

In [117]:
kge(predictions.y[0].view(-1, 1).squeeze(),predictions.output.squeeze())

tensor(0.9767)